In [51]:
import sys
from pathlib import Path
from subprocess import call

In [142]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from glob import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import MeCab
from wordcloud import WordCloud
import re
import os
import urllib.request

In [143]:
def convert_pdf_to_txt(path): # 引数にはPDFファイルパスを指定
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    laparams.detect_vertical = True # Trueにすることで綺麗にテキストを抽出できる
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    maxpages = 0
    caching = True
    pagenos=set()
    fstr = ''
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,caching=caching, check_extractable=True):
        interpreter.process_page(page)

        str = retstr.getvalue()
        fstr += str

    fp.close()
    device.close()
    retstr.close()
    return fstr

In [144]:
path = "documents/JPA-1999069893.pdf"

In [145]:
mecab = MeCab.Tagger ("-Owakati")
text = convert_pdf_to_txt(path).replace('\n', '')
text = mecab.parse (text)
pattern = r"[0-9A-Za-zぁ-んァ-ヴ一-龠]"
stopwords = re.sub(pattern, '',text)
# print(stopwords)

def wakati_by_mecab(text):
    tagger = MeCab.Tagger('')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

def lower_text(text):
    return text.lower()

def normalize_number(text):
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '', text)
    return replaced_text

def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(path):
    stop_words = []
    for w in open(path, "r"):
        w = w.replace('\n','')
        if len(w) > 0:
          stop_words.append(w)
    return stop_words    

file_path = "/Users/yusuke-n/code/work/read_pdf/stopword.txt"
# text = wakati_by_mecab(text)
# text = lower_text(text)
# text = normalize_number(text)
download_stopwords(file_path)
stop_words = create_stopwords(file_path)

File already exists.


In [146]:
from sklearn.feature_extraction.text import CountVectorizer
comments = ["あなたは春が好きね", text]
sample_dict = {"comment": comments} 
samples = pd.DataFrame.from_dict(sample_dict)
samples["target"] = samples["comment"].apply(wakati_by_mecab).apply(lower_text).apply(normalize_number)

count_vectorizer = CountVectorizer(stop_words=stop_words,token_pattern=u'(?u)\\b\\w+\\b')
feature_vectors = count_vectorizer.fit_transform(samples['target'])
vocabulary = count_vectorizer.get_feature_names() 

In [147]:
from wordcloud import WordCloud
wordcloud = WordCloud(
                      font_path="/system/Library/Fonts/ヒラギノ角ゴシック W4.ttc",
                      colormap='copper_r',
                      width=800,height=600).generate(samples['target'][1])

In [148]:
wordcloud.to_file("./wordcloud_sample.png")